## Creating Dim Customer table

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from delta.tables import DeltaTable
src_df=spark.read.format("delta").load('abfss://silver@telcostoragelayer.dfs.core.windows.net/customer/')
src_df.display()

In [0]:
#createing dim table schema
df_sink=spark.sql('''
select 1 as dim_cust_key, customer_id,first_name,last_name,gender,dob,phone_number,email,timestamp from delta.`abfss://silver@telcostoragelayer.dfs.core.windows.net/customer` where 1=0''')
df_sink.display()


## dim customer sink initial and incremental

In [0]:
if spark.catalog.tableExists('telco_catalog.gold.dim_customer'):
  df_sink=spark.sql('''
    select dim_customer_key,customer_id,first_name,last_name,gender,dob,phone_number,email,timestamp from telco_catalog.gold.dim_customer''')

else:
  df_sink=spark.sql('''
    select 1 as dim_customer_key,customer_id,first_name,last_name,gender,dob,phone_number,email,timestamp from delta.`abfss://silver@telcostoragelayer.dfs.core.windows.net/customer` where 1=0''')


In [0]:
df_filter=src_df.join(df_sink,src_df.customer_id==df_sink.customer_id,'left').select(src_df.customer_id,src_df.first_name,src_df.last_name,src_df.gender,src_df.dob,src_df.phone_number,src_df.email,src_df.timestamp,df_sink.dim_customer_key.alias('dim_customer_key'))
df_filter.display()

In [0]:
df_filter_old=df_filter.filter(df_filter.dim_customer_key.isNotNull()).select('customer_id','first_name','last_name','gender','dob','phone_number','email','timestamp','dim_customer_key')
df_filter_old.display()


In [0]:
df_filter_new=df_filter.filter(df_filter.dim_customer_key.isNull()).select('customer_id','first_name','last_name','gender','dob','phone_number','email','timestamp','dim_customer_key')
df_filter_new.display()

In [0]:
if not spark.catalog.tableExists('telco_catalog.gold.dim_customer'):
    max_value = 0
else:
    result = spark.sql('SELECT MAX(dim_customer_key) FROM telco_catalog.gold.dim_customer').collect()[0][0]
    max_value = result if result is not None else 0  # handle case when table exists but is empty
    max_timestamp = spark.sql('SELECT MAX(timestamp) FROM telco_catalog.gold.dim_customer').collect()[0][0]
# Step 2: Assign row_number (you can order by something meaningful like customer_id or timestamp)
windowSpec = Window.orderBy("customer_id")  # Or use timestamp if needed
df_filter_new = df_filter_new.withColumn("row_num", row_number().over(windowSpec))

# Step 3: Assign new surrogate keys
df_filter_new = df_filter_new.withColumn("dim_customer_key", col("row_num") + lit(max_value)).drop("row_num")

# Final output
df_filter_new.display()

## create a final df= df_filter_old+df_filter_new

In [0]:
df_final=df_filter_old.union(df_filter_new)
df_final=df_final.filter(df_final.timestamp>max_timestamp)
df_final.display()


In [0]:

if spark.catalog.tableExists('telco_catalog.gold.dim_customer'):
  delta_tbl=DeltaTable.forPath(spark,'abfss://gold@telcostoragelayer.dfs.core.windows.net/dim_customer')
  delta_tbl.alias('tgt').merge(df_final.alias('src'),'tgt.customer_id=src.customer_id').whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
#initial run
else:
    df_final.write.format('delta').mode('overwrite').option('path','abfss://gold@telcostoragelayer.dfs.core.windows.net/dim_customer').saveAsTable('telco_catalog.gold.dim_customer')